## Acoustic Integration - Automatic Differentiation

http://localhost:8888/?token=sloth

In [ ]:
# from tqdm import trange
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib ipympl

import drjit as dr
import mitsuba as mi

from libs import utils, acoustic_torch

mi.set_variant('cuda_ad_acoustic')
mi.set_log_level(mi.LogLevel.Warn)

plt.style.use('ggplot')
utils.drjit_turn_off_optimizations(False)

sess_seed   = np.random.randint(0, 2**30)
sess_seed_g = np.random.randint(0, 2**30)
print(f"session seeds are: sess_seed={sess_seed}; sess_seed_g={sess_seed_g}")

### Scene Construction

In [ ]:
absorption = 0.5
scattering = [0.5, 0.75]

config = {
    "box_dim":     [24., 12., 6.],
    "mic_pos":     [ 4.,  2., 3.],
    "speaker_pos": [20.,  2., 3.],
    "speaker_radius": 0.5, #0.1,

    "absorption": absorption,
    "scattering": [(i + 1, s) for i, s in enumerate(scattering)],

    "wav_bins":  len(scattering), # x
    "time_bins":  40,             # y
    "max_time":   0.2,

    # "integrator": "acousticpath",
    "integrator": "prb_acoustic",
    # "integrator": "prb_reparam_acoustic",
    "max_depth": 50,
    "spp": 2**18,
}

fs = config["time_bins"] / config["max_time"]
time = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)

# config["max_depth"] = utils.estimate_max_depth(config["box_dim"], config["max_time"], 1.5)
print(f"max_depth = {config['max_depth']}")

scene_dict = utils.shoebox_scene(**config)

scene_dict["integrator"]["skip_direct"] = False

scene_dict["shoebox"]["top"]["bsdf"] = {
    "type": "acousticbsdf",
    "scattering": { "type": "spectrum", "value": config['scattering'] },
    "absorption": 0.2,
}

scene_dict["shoebox"]["bottom"]["bsdf"] = {
    "type": "acousticbsdf",
    "scattering": { "type": "spectrum", "value": config['scattering'] },
    "absorption": 0.8,
}

scene = mi.load_dict(scene_dict)

### Reference Histogram

In [ ]:
hist_ref = mi.render(scene, seed=sess_seed)
utils.plot_hist(hist_ref[:, :, 0], **config)

### Optimization Setup

In [ ]:
key1 = "shoebox.top.bsdf.absorption.value"
key2 = "shoebox.bottom.bsdf.absorption.value"
params = mi.traverse(scene)
display(params)
display(params[key1])
display(params[key2])

In [ ]:
params_ref   = mi.Float([params[key1][0], params[key2][0]])
params[key1] = mi.Float(0.6)
params[key2] = mi.Float(0.4)

opt = mi.ad.Adam(lr=0.005)
opt[key1] = params[key1]
opt[key2] = params[key2]
params.update(opt);

In [ ]:
def loss(a, b):
    # a     = acoustic_torch.EDC(a[:, :, 0], db=True, norm=True)
    # b     = acoustic_torch.EDC(b[:, :, 0], db=True, norm=True)
    return utils.mse(a, b)

In [ ]:
hist = mi.render(scene, seed=sess_seed)
dr.enable_grad(hist)
l = loss(hist, hist_ref)
dr.backward(l)
grad = dr.grad(hist)
utils.plot_hist(grad[:, :, 0], **config)

### Main Loop

In [ ]:
vals, losses = [], []
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.set_xlim(-1, 51)
ax1.set_ylim(-0.1, 1.1)
ax2.set_xlim(-1, 50)
ax2.set_ylim(-0.1, 1.1);

In [ ]:
iters  = 400
if iters > 1:
    n  = len(vals) + iters

# for i in trange(iters) if iters > 2 else range(iters):
for i in range(iters):
    hist = mi.render(scene, params, seed=sess_seed, seed_grad=sess_seed_g)
    l    = loss(hist, hist_ref)
    dr.backward(l)

    if iters < 1:
        display(dr.grad(opt[key1]))
        display(dr.grad(opt[key2]))
        dr.set_grad(opt[key1], 0)
        dr.set_grad(opt[key2], 0)
    else:
        opt.step()
        opt[key1] = dr.clamp(opt[key1], 0.0, 1.0)
        opt[key2] = dr.clamp(opt[key2], 0.0, 1.0)
        params.update(opt)

        vals.append(np.array([params[key1], params[key2]])[:, 0])
        losses.append(l.numpy()[0])

        ax1.clear()
        ax1.plot(vals)
        ax1.hlines([0.8, 0.2], 0, n, colors='k', linestyles='dotted')
        ax1.set_xlim(-n * 0.02, n * 1.02)
        ax1.set_ylim(-0.1, 1.1)
        ax2.clear()
        ax2.plot(losses)
        ax2.set_xlim(-1, n)
        fig.canvas.draw()

In [ ]:
if iters > 1:
    losses = np.array(losses)
    vals   = np.array(vals)
    data   = np.hstack([losses[:, None], vals])
    # np.save("/home/daniel/diff-absorption-roof-ceiling.npy", data)